In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

# Preliminary functions
Functions that make our life easier, since we will be reusing some code.

In [2]:
def dimensions(molecule):
    """
    Return the dimensions (N, M) of a molecule in a given basis. Here, N is the number of electrons and M the number of spinors.
    """
    N = molecule.numberOfElectrons()

    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    M = spin_orbital_basis.numberOfSpinors()
    
    return (N, M)

In [3]:
def onv_bases(N, K, transition = "detachment"):
    """
    Create two ONV bases, based on whether we are describing an electron attachment or detachment.
    """

    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2
    
    if transition == "detachment":
        onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta - 1)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        
    elif transition == "attachment":
        onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha + 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        
    else:
        print("Specify a valid electron transition process: attachment/detachment.")
        return None
    
    return (onv_basis_I, onv_basis_J)

In [4]:
def C_RHF(molecule):
    
    N = molecule.numberOfElectrons()
    N_P = molecule.numberOfElectronPairs()
    
    spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spin_orbital_basis.numberOfSpatialOrbitals()
    #print("N: {}\nN_P: {}\nK: {}".format(N, N_P, K))

    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
    S = spin_orbital_basis.quantizeOverlapOperator().parameters()

    environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
    solver = gqcpy.RHFSCFSolver.Plain()
    objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
    rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
    
    return rhf_parameters.expansion()

In [5]:
def C_UHF(molecule):
    N = molecule.numberOfElectrons()
    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2

    spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spinor_basis.numberOfSpatialOrbitals()
    #print("N_alpha: {}\nN_beta: {}\nK: {}".format(N_alpha, N_beta, K))
    
    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule) 
    S = spinor_basis.quantizeOverlapOperator().parameters()
    
    environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S)
    solver = gqcpy.UHFSCFSolver.Plain()
    uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
    return uhf_parameters.expansion()

In [6]:
def TM(C_ion, C_neutral):
    return C_ion.inverse().transformed(C_neutral)

In [7]:
def uFCI(molecule, onv_basis):
    """
    FCI calculations for an unrestricted spin-orbital basis.
    """
    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G") # the underlying scalar AO basis is not orthonormal
    spin_orbital_basis.lowdinOrthonormalize() # after using Löwdin's orthonormalizing procedure, the spin orbital basis is now orthonormal
    
    hamiltonian = gqcpy.USQHamiltonian.Molecular(spin_orbital_basis, molecule)
    
    solver = gqcpy.EigenproblemSolver.Dense()
    environment = gqcpy.CIEnvironment.Dense(hamiltonian, onv_basis)

    ci_parameters = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
    
    return ci_parameters

# Molecular set-up

Generate the neutral molecule with N electrons, a positive ion with N-1 electrons (charge +1) and a negative ion with N+1 electrons (charge -1).

In [8]:
#neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , 0)  # create a neutral molecule with N electrons
#positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , +1)  # create a an ion N-1 electrons
#negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , -1)  # create a an ion with N+1 electrons

neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule with N electrons
positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion N-1 electrons
negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , -1)  # create a an ion with N+1 electrons

In [9]:
N, M = dimensions(neutral_molecule)
K = M//2 # number of spinors per spin compound

onv_basis_I, onv_basis_J = onv_bases(N, K, "detachment")

# Fukui matrix in the uncorrelated limit

In [10]:
C_neutral = C_UHF(neutral_molecule)
C_ion = C_UHF(positive_ion)

In [11]:
U = TM(C_ion, C_neutral)

In [12]:
print("Is C_ion unitary? ", C_ion.isUnitary())
print("Is C_neutral unitary? ", C_neutral.isUnitary())
print("Is U unitary? ", U.isUnitary())

Is C_ion unitary?  False
Is C_neutral unitary?  False
Is U unitary?  True


In [13]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_I)
print(Psi_I.coefficients())
Psi_J = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_J)
print(Psi_J.coefficients())

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [14]:
DM_I = Psi_I.calculateSpinResolved1DM()
DM_J = Psi_J.calculateSpinResolved1DM()

In [15]:
DM_I = DM_I.transformed(U)

fukui_matrix_neg = (DM_J - DM_I).orbitalDensity()
fukui_eigenvalues, fukui_naturals = np.linalg.eigh(fukui_matrix_neg)
np.round(fukui_naturals,3)

array([[-0.001, -0.   ,  1.   , -0.023, -0.   , -0.002,  0.   ],
       [-0.23 , -0.   ,  0.021,  0.933, -0.   , -0.275,  0.   ],
       [ 0.   , -0.653,  0.   , -0.   , -0.758,  0.   ,  0.   ],
       [ 0.607,  0.   ,  0.01 ,  0.358,  0.   ,  0.709,  0.   ],
       [ 0.   , -0.   , -0.   , -0.   , -0.   ,  0.   , -1.   ],
       [ 0.761,  0.   ,  0.   , -0.004, -0.   , -0.649,  0.   ],
       [-0.   ,  0.758,  0.   ,  0.   , -0.653,  0.   ,  0.   ]])

In [16]:
fukui_eigenvalues

array([-2.98678643e-01, -2.74420688e-01, -6.65719147e-16,  7.44488336e-04,
        2.74420688e-01,  2.97934154e-01,  1.00000000e+00])

# Fukui matrix when correlation is present

In [17]:
Psi_I = uFCI(positive_ion, onv_basis_I)
print(Psi_I.coefficients())
Psi_J = uFCI(neutral_molecule, onv_basis_J)
print(Psi_J.coefficients())

[ 3.61649356e-01  1.55369378e-14  5.94350654e-15 -1.07639598e-15
 -5.33938256e-16  1.97628951e-01 -2.57677851e-01  1.28332744e-01
  1.57317256e-02 -1.10727526e-14  6.24311438e-15  6.98562584e-16
  8.83939358e-17 -3.88969318e-17  3.10687705e-17  1.97628951e-01
  2.57677851e-01  1.28332744e-01  1.57317256e-02  6.38563402e-15
  5.19353000e-15  6.23177983e-16  2.25522927e-15  2.72798510e-16
  1.15278135e-17  2.42808960e-01  8.80263378e-16  1.01068487e-16
 -1.67070089e-01 -1.97178487e-02  2.23075186e-17 -6.14258492e-15
 -7.26702108e-16 -6.11960707e-18  1.44654341e-17 -6.85984847e-17
  1.49653519e-02 -2.14172008e-02  1.55017701e-02  2.02374095e-03
 -3.01830711e-16 -2.40084155e-17  1.92813615e-17  3.25284420e-18
  5.00065882e-04 -1.70979305e-03 -2.59530773e-04  1.63543568e-03
  2.69626481e-04 -5.83321441e-05 -1.58667935e-16 -8.06586834e-17
 -8.56296851e-19  1.48909023e-18  1.82330563e-02 -2.10697273e-03
 -3.18788675e-04 -1.59333550e-02 -1.97161957e-03 -4.14749546e-05
 -1.96586044e-16 -4.88415

In [18]:
DM_I = Psi_I.calculateSpinResolved1DM()
DM_J = Psi_J.calculateSpinResolved1DM()

In [19]:
DM_I = DM_I.transformed(U)

fukui_matrix_neg = (DM_J - DM_I).orbitalDensity()
fukui_eigenvalues, fukui_naturals = np.linalg.eigh(fukui_matrix_neg)
np.round(fukui_naturals,3)

array([[ 0.057,  0.01 ,  0.994,  0.061,  0.013,  0.051,  0.036],
       [-0.495, -0.136,  0.106, -0.624, -0.099, -0.456, -0.344],
       [-0.163, -0.629,  0.001,  0.603,  0.223, -0.339, -0.223],
       [-0.627,  0.367,  0.003,  0.132,  0.613,  0.279, -0.03 ],
       [ 0.05 , -0.005, -0.001, -0.134,  0.357, -0.521,  0.762],
       [ 0.36 ,  0.582,  0.003,  0.178,  0.196, -0.52 , -0.437],
       [ 0.448, -0.335, -0.008, -0.42 ,  0.631,  0.233, -0.242]])

In [20]:
fukui_eigenvalues

array([-1.52805182e+00, -6.97471559e-01,  5.12927886e-05,  1.49626364e-01,
        7.02979684e-01,  8.78279811e-01,  1.49458623e+00])